## Set up

In [29]:
# Python version

!python --version

Python 3.11.3


In [30]:
# Install required packages

!pip install pandas==2.2.3 rapidfuzz==3.13.0


[notice] A new release of pip is available: 23.2.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


In [31]:
# Imports

import warnings
import logging
import pandas as pd
from pathlib import Path
from urllib.parse import urlparse
from rapidfuzz import fuzz as rfuzz
import time
import os

In [32]:
# Set up logging

logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    force=True,  # for logging to work in Notebooks
)

In [33]:
# Ignore warnings

warnings.filterwarnings('ignore', category=RuntimeWarning)

In [34]:
# Set minimum similarity scores to consider a pair of products as duplicates

NAME_SCORE_THRESHOLD = 85.0
URL_SCORE_THRESHOLD = 95.0

## Data preparation

In [35]:
# Read all csv files in input folder

input_dir = Path('input')
csv_files = list(input_dir.glob('*.csv'))

logging.info('Reading files...')

dfs = [pd.read_csv(file) for file in csv_files]

logging.info(f'Read {len(dfs)} files successfully.')

2025-06-10 21:21:05,122 - INFO - Reading files...
2025-06-10 21:21:06,730 - INFO - Read 2 files successfully.


In [36]:
# Standardise column names across all data frames

for df in dfs:
    
    if 'product_name' not in df.columns:
        df.rename(columns={'name': 'product_name'}, inplace=True)

    if 'seller_website' not in df.columns:
        df.rename(columns={'url': 'seller_website'}, inplace=True)

    if 'product_category' not in df.columns:
        df.rename(columns={'category': 'product_category'}, inplace=True)
        df.rename(columns={'main_category': 'product_category'}, inplace=True)

In [37]:
# Drop unused columns

columns_to_keep = ['product_name', 'seller_website', 'description', 'product_category']
dfs = [df[columns_to_keep] for df in dfs]

In [38]:
# Concatenate all data frames into one

df = pd.concat(dfs, ignore_index=True)

In [39]:
# Helper function to clean text

def clean_text(series: pd.Series) -> pd.Series:
    """
    Lowercase text, remove leading/trailing and multiple whitespaces, and remove specific Unicode control characters.
    """
    unicode_chars = r'[\u202a-\u202e\u200b-\u200f]'
    return (series
            .str.lower()
            .str.strip()
            .str.replace(r'\s+', ' ', regex=True)
            .str.replace(unicode_chars, '', regex=True))

In [40]:
# Helper function to clean website URLs

def normalize_url(url: str) -> str:
    """
    Normalize URL:
    - Remove URL scheme (http:// or https://)
    - Remove 'www.' if present
    - Lowercase domain
    - Remove trailing slashes (/)
    """
    if pd.isna(url):
        return url
        
    parsed = urlparse(url)
    
    netloc = parsed.netloc.lower()
    if netloc.startswith('www.'):
        netloc = netloc[4:]
    
    path = parsed.path.rstrip('/')
    
    return netloc + path

In [41]:
# Apply cleaning functions

df['product_name_c'] = clean_text(df['product_name'])
df['seller_website_c'] = df['seller_website'].apply(normalize_url)
df['description_c'] = clean_text(df['description'])
df['product_category_c'] = clean_text(df['product_category'])

In [42]:
# Remove exact duplicates using product name and seller website separately

for col in ['product_name_c', 'seller_website_c']:
    
    before = len(df)
    df = df[(~df.duplicated(subset=[col])) | (df[col].isnull())]  # ensure missing values don't get dropped
    after = len(df)
    
    logging.info(f'Dropped {before - after} exact duplicates based on column: {col}')

2025-06-10 21:21:09,267 - INFO - Dropped 10977 exact duplicates based on column: product_name_c
2025-06-10 21:21:09,306 - INFO - Dropped 19223 exact duplicates based on column: seller_website_c


In [43]:
# Sort alphabetically based on product name

df.sort_values(by='product_name_c', ignore_index=True, inplace=True)

In [44]:
# Examine cleaned data frame

df.head()

,product_name,seller_website,description,product_category,product_name_c,seller_website_c,description_c,product_category_c
0,#Back (HashBack),https://hashback.app/,"#Back for record your screen, share it with yo...",Screen and Video Capture Software,#back (hashback),hashback.app,"#back for record your screen, share it with yo...",screen and video capture software
1,#DIESE,https://diesesoftware.com/,"#DIESE is accessible from any web browser, and...",Performing Arts Software,#diese,diesesoftware.com,"#diese is accessible from any web browser, and...",performing arts software
2,#paid,https://www.hashtagpaid.com/,#paid is a creator marketplace that connects v...,Social Media Marketing Software,#paid,hashtagpaid.com,#paid is a creator marketplace that connects v...,social media marketing software
3,#stratapp,https://stratappsaas.com/,#stratapp is a simple management app where ind...,Objectives and Key Results (OKR) Software,#stratapp,stratappsaas.com,#stratapp is a simple management app where ind...,objectives and key results (okr) software
4,&facts,https://www.andfacts.com/,&facts gives brands access to market insights ...,Market Intelligence Software,&facts,andfacts.com,&facts gives brands access to market insights ...,market intelligence software


## Fuzzy matching

In [45]:
# Helper function to generate a pandas df of combinations

def generate_combinations_df(df, n=5):
    """
    Generate pairwise combinations of rows from the input DataFrame.

    For each row, combinations are generated with the n preceding and n following
    rows, avoiding self-combinations and duplicate pairs.

    Args:
        df (pd.DataFrame): DataFrame where first column (typically product name) is
            alphabetically sorted and free of exact duplicates.
        window_size (int): Number of rows above and below to consider for combinations.

    Returns:
        combinations_df (pd.DataFrame): DataFrame containing all unique combinations of rows
        within window range, with columns from both rows suffixed with _1 and _2.
    """
    rows = [tuple(row) for row in df.itertuples(index=False)]
    combinations = set()

    # Generate combinations
    for i, row in enumerate(rows):
    
        start = max(0, i - n)
        end = min(len(rows), i + n + 1)
        neighbours = rows[start:end]
    
        for neighbour in neighbours:
            if row[0] != neighbour[0]:  # avoid self-combinations
                if row[0] < neighbour[0]:  # order combinations alphabetically
                    combination = (row, neighbour)
                else:
                    combination = (neighbour, row)
                combinations.add(combination)

    # Organise results into array
    data_for_df = []
    for row1, row2 in combinations:
        combined_row = list(row1) + list(row2)
        data_for_df.append(combined_row)

    # Create list of new column names
    original_columns = df.columns.tolist()
    new_columns = []
    for col in original_columns:
        new_columns.append(f'{col}_1')
    for col in original_columns:
        new_columns.append(f'{col}_2')

    # Store combinations into data frame
    combinations_df = pd.DataFrame(data_for_df, columns=new_columns)

    # Sort using first column and reset index
    sort_cols = [f'{original_columns[0]}_1', f'{original_columns[0]}_2']
    combinations_df.sort_values(by=sort_cols, inplace=True)
    combinations_df.reset_index(drop=True, inplace=True)
    
    return combinations_df

In [46]:
# Generate combinations of software products for later comparison

columns_to_keep = [
    'product_name_c',
    'seller_website_c',
    'description_c',
    'product_category_c',
]

combinations_df = generate_combinations_df(df[columns_to_keep])

In [47]:
# Helper function to compute similarity score between two strings

def safe_similarity(a, b):
    """
    Compute fuzzy token sort ratio similarity between two values safely.
    Handles missing values by returning None if either input is NaN.

    Args:
        a (str or NaN): First string to compare.
        b (str or NaN): Second string to compare.

    Returns:
        int or None: Similarity score (0-100) or None if either input is missing.
    """
    if pd.isna(a) or pd.isna(b):
        return None
    return rfuzz.token_sort_ratio(str(a), str(b))

In [48]:
# Helper function to add similarity score between two columns to df

def add_similarity_column(df, columns):
    """
    Add a new column to the DataFrame with fuzzy similarity scores between two existing columns.

    Args:
        df (pd.DataFrame): The DataFrame containing the data.
        columns (tuple): A tuple of (col1, col2, out_col) where:
            - col1 (str): Name of the first input column.
            - col2 (str): Name of the second input column.
            - out_col (str): Name of the output column to store similarity scores.

    Returns:
        pd.DataFrame: The DataFrame with the new similarity column added.
    """
    col1, col2, out_col = columns
    df[out_col] = df.apply(lambda row: safe_similarity(row[col1], row[col2]), axis=1)
    return df

In [49]:
# Compute similarity scores between pairs of products and time processing

start = time.time()

logging.info('Computing similarity between products...')

columns_to_compare = [
    ('product_name_c_1', 'product_name_c_2', 'name_similarity'),
    ('seller_website_c_1', 'seller_website_c_2', 'url_similarity'),
    ('description_c_1', 'description_c_2', 'description_similarity'),
    ('product_category_c_1', 'product_category_c_2', 'category_similarity'),
]

for columns in columns_to_compare:
    add_similarity_column(combinations_df, columns)

end = time.time()

logging.info(f'{len(combinations_df)} rows processed in {(end - start):.1f} seconds')

2025-06-10 21:21:11,922 - INFO - Computing similarity between products...
2025-06-10 21:21:27,432 - INFO - 389845 rows processed in 15.5 seconds


In [50]:
# Get a list of duplicated products

duplicates = combinations_df[
    (combinations_df['name_similarity'] > NAME_SCORE_THRESHOLD) &
    ((combinations_df['url_similarity'].isna()) | (combinations_df['url_similarity'] > URL_SCORE_THRESHOLD))
]

logging.info(f'Found {len(duplicates)} product duplicates based on fuzzing matching scores.')

2025-06-10 21:21:27,450 - INFO - Found 1766 product duplicates based on fuzzing matching scores.


In [51]:
# Examine duplicates

columns_to_display = [
    'product_name_c_1',
    'product_name_c_2',
    'name_similarity',
    'seller_website_c_1',
    'seller_website_c_2',
    'url_similarity',
]

duplicates[columns_to_display].head(10)

,product_name_c_1,product_name_c_2,name_similarity,seller_website_c_1,seller_website_c_2,url_similarity
67,.net core,.netcore,94.117647,docs.microsoft.com/en-us/dotnet/core,NaN,NaN
245,1010 data,1010data,94.117647,1010data.com,NaN,NaN
1813,3d product configurator,3d source product configurator,86.792453,cl3ver.com,NaN,NaN
2405,4mat,4mata,88.888889,NaN,4mata.com,NaN
2500,4screen,4screens,93.333333,4screen.com,NaN,NaN
3165,a cloud guru,a cloud guru lms,85.714286,acloud.guru,NaN,NaN
4785,accelo,accelops,85.714286,accelo.com,NaN,NaN
5364,accountberry,accounter,85.714286,accountberry.com,NaN,NaN
6726,actian,actiance,85.714286,NaN,NaN,NaN
6930,actionvfx,actionx,87.500000,actionvfx.com,NaN,NaN


In [52]:
# Drop duplicates based on fuzzy matching results

to_drop = list(duplicates['product_name_c_2'])
deduplicated_df = df[~df['product_name_c'].isin(to_drop)]

logging.info(f'Removed {len(to_drop)} duplicates based on fuzzy matching scores.')

2025-06-10 21:21:27,529 - INFO - Removed 1766 duplicates based on fuzzy matching scores.


## Save results

In [53]:
# Helper function to save file to CSV

def save_to_csv(df, file_path, columns_to_keep):
    """Save a DataFrame to a CSV, keeping only specified columns."""
    
    os.makedirs(os.path.dirname(file_path), exist_ok=True)
    df.to_csv(file_path, index=False, columns=columns_to_keep)
    
    logging.info(f'File successfully saved to {file_path}')

In [54]:
# Save product catalog to csv

scores_columns = [
    'product_name_c_1',
    'product_name_c_2',
    'name_similarity',
    'seller_website_c_1',
    'seller_website_c_2',
    'url_similarity',
    'description_c_1',
    'description_c_2',
    'description_similarity',
    'product_category_c_1',
    'product_category_c_2',
    'category_similarity',
]

catalogue_columns_to_keep = [
    'product_name',
    'seller_website',
    'description',
    'product_category',
]

duplicates = pd.DataFrame(duplicates)

save_to_csv(duplicates, 'output/duplicates.csv', scores_columns)
save_to_csv(combinations_df, 'output/scores.csv', scores_columns)
save_to_csv(df, 'output/duplicated_product_catalogue.csv', catalogue_columns_to_keep)
save_to_csv(deduplicated_df, 'output/master_product_catalogue.csv', catalogue_columns_to_keep)

2025-06-10 21:21:27,586 - INFO - File successfully saved to output/duplicates.csv
2025-06-10 21:21:34,866 - INFO - File successfully saved to output/scores.csv
2025-06-10 21:21:35,459 - INFO - File successfully saved to output/duplicated_product_catalogue.csv
2025-06-10 21:21:36,041 - INFO - File successfully saved to output/master_product_catalogue.csv


## Limitations

In [55]:
rfuzz.token_sort_ratio('amazon ses', 'amazon simple email service (ses)')

46.51162790697675

In [56]:
rfuzz.token_sort_ratio('.net', '.net 4.5')

66.66666666666667

In [57]:
rfuzz.token_sort_ratio(
    '1-Click Ready Tool Windows process heat map on Windows 2012 R2',
    '1-Click Ready Windows Tool Weka on Windows 2012 R2',
)

80.35714285714286